In [1]:
import os
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

In [2]:
os.environ['PATH'] += r"C:\SeleniumDrivers\chromedriver-win64\chromedriver-win64"
options = Options()
options.add_argument("--headless")
driver = webdriver.Chrome(options=options)

In [3]:
SEASONS = list(range(2016, 2024))

In [4]:
# getting references to those folders and a place to store
DATA_DIR = "data"
STANDINGS_DIR = os.path.join(DATA_DIR, "standings")
SCORES_DIR = os.path.join(DATA_DIR, "scores")

In [5]:
# get the html

async def get_html(url, selector, sleep=5, retries=3):
    html = None
    for i in range(1, retries+1):
        time.sleep(sleep * i)
        try:
                #initializes playwright instance
                # waits until browser is done launching
                driver.get(url)
                #grab the specific html
                element = driver.find_element(By.XPATH, selector)
                print(f"found element at {url}")
                html = element.get_attribute('innerHTML')
        except:
            print(f"Error on {url}")
            continue
        # if there are no errors run t
        else:
            break
    return html


In [6]:
#scraping seasons
async def scrape_season(season):
    url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
    #find the id element of content in the html, followed by the html that has filter as the id
    html = await get_html(url, "//div[@class='filter']")
    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    href = [l["href"] for l in links]
    standing_pages = [f"https://www.basketball-reference.com{l}" for l in href]
    print(standing_pages)
    for url in standing_pages:
        save_path = os.path.join(STANDINGS_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue
        html = await get_html(url, "//div[@id='div_schedule']")
        with open(save_path, "w+") as f:
            f.write(html)



In [7]:
#scraping all seasons
# for season in SEASONS:
#     await scrape_season(season)

In [8]:
standings_files = os.listdir(STANDINGS_DIR)

In [9]:
async def scrape_game(standings_file):
    with open(standings_file, 'r') as f:
        html = f.read()
    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    hrefs = [l.get("href") for l in links]
    box_scores = [l for l in hrefs if l and "boxscore" in l and ".html" in l]
    box_scores = [f"https://www.basketball-reference.com{l}" for l in box_scores]
    
    for url in box_scores:
        save_path = os.path.join(SCORES_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue
        html = await get_html(url, "//div[@id='content']")
        if not html:
            continue
        with open(save_path, "w+", encoding='utf-8') as f:
            f.write(html)

In [10]:
standings_files = [s for s in standings_files if ".html" in s]

In [11]:
for f in standings_files:
    filepath = os.path.join(STANDINGS_DIR, f)
    await scrape_game(filepath)

found element at https://www.basketball-reference.com/boxscores/202212210CLE.html
found element at https://www.basketball-reference.com/boxscores/202212210PHI.html
found element at https://www.basketball-reference.com/boxscores/202212210ATL.html
found element at https://www.basketball-reference.com/boxscores/202212210BOS.html
found element at https://www.basketball-reference.com/boxscores/202212210BRK.html
found element at https://www.basketball-reference.com/boxscores/202212210NYK.html
found element at https://www.basketball-reference.com/boxscores/202212210HOU.html
found element at https://www.basketball-reference.com/boxscores/202212210MIN.html
found element at https://www.basketball-reference.com/boxscores/202212210OKC.html
found element at https://www.basketball-reference.com/boxscores/202212210SAC.html
found element at https://www.basketball-reference.com/boxscores/202212210LAC.html
found element at https://www.basketball-reference.com/boxscores/202212220NOP.html
found element at